In [2]:
import numpy as np 
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
import os
from tl_tools import *

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

In [3]:
setup_mixed_precision()

path = '/Users/pimpijnenburg/Desktop/Thesis/USTC_SmokeRS_dataset/data/created_data/train_val_test_split'
train, val, test = load_and_prepare_data(path)

Mixed precision policy set to: mixed_float16


2024-10-16 10:23:34.978021: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3 Pro
2024-10-16 10:23:34.978054: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 18.00 GB
2024-10-16 10:23:34.978062: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 6.00 GB
2024-10-16 10:23:34.978076: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-10-16 10:23:34.978089: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [4]:
for images, labels in train.take(1):
    print("Train Image shape:", images.shape)
    print("Train Label shape:", labels.shape)
    print("Train Image min-max:", tf.reduce_min(images).numpy(), tf.reduce_max(images).numpy())
    print("Train Unique labels:", np.unique(labels))
    print(labels[:2])

print()

for images, labels in val.take(1):
    print("Val Image shape:", images.shape)
    print("Val Label shape:", labels.shape)
    print("Val Image min-max:", tf.reduce_min(images).numpy(), tf.reduce_max(images).numpy())
    print("Val Unique labels:", np.unique(labels))
    print(labels[:2])

Train Image shape: (32, 256, 256, 3)
Train Label shape: (32, 6)
Train Image min-max: 0.0 1.0
Train Unique labels: [0. 1.]
tf.Tensor(
[[0. 0. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0. 0.]], shape=(2, 6), dtype=float32)

Val Image shape: (32, 256, 256, 3)
Val Label shape: (32, 6)
Val Image min-max: 0.0 1.0
Val Unique labels: [0. 1.]
tf.Tensor(
[[0. 0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0. 0.]], shape=(2, 6), dtype=float32)


2024-10-16 10:24:15.027995: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-10-16 10:24:15.055256: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [4]:
base_model = tf.keras.applications.EfficientNetV2S(include_top= False, weights = None, input_shape= (256, 256,3))

x = base_model.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(1024, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01))(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(6, activation='softmax', dtype='float32')(x)

efficientnet = Model(inputs=base_model.input, outputs=x, name='EfficientNetV2S')

In [5]:
optimizer = create_optimizer(learning_rate= 1e-3)
efficientnet.compile(optimizer=optimizer, 
                     loss='categorical_crossentropy', 
                     metrics=['accuracy', 'F1Score'])

early_stopping = EarlyStopping(
    monitor='val_loss',  
    patience=5,          
    restore_best_weights=True,  
    start_from_epoch=20  
)

In [6]:
history = history = efficientnet.fit(train, validation_data=val, epochs=50, verbose=1, callbacks= early_stopping)

Epoch 1/50


2024-10-16 10:23:04.307757: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


: 

: 

In [ ]:
efficientnet.save('efficientnetv2_full_model_v1.h5')